<a href="https://colab.research.google.com/github/cristinarainich/Sternberg_analysis/blob/main/Creating_data_frames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import pylab as pl
import scipy.io
from pathlib import Path

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
working_path = Path('/content/gdrive/MyDrive/CoganLab')

In [5]:
# controls' data
eeg = scipy.io.loadmat(
    working_path.joinpath('Subject_NeighborhoodSternberg_EEG_060322.mat')
    )
# patients' data
ieeg = scipy.io.loadmat(
    working_path.joinpath('Subject_NeighborhoodSternberg_IEEG_060322.mat')
    )
new_subject1 = scipy.io.loadmat(
    working_path.joinpath('D72_Block_1_TrialData.mat')
    )
new_subject2 = scipy.io.loadmat(
    working_path.joinpath('D73_Block_1_TrialData.mat')
    )

## iEEG dataset 

In [6]:
data_ieeg = ieeg['Subject'] # contains trials for each of the subjects
print('Number of subjects in EEG dataset is', data_ieeg.shape[1])

Number of subjects in EEG dataset is 30


In [7]:
subjects_ieeg = [] # subject numbers
for i in data_ieeg['Name'][0]:
  subjects_ieeg.append(i[0])

In [8]:
def get_to_df(data):
  # this function reads into .mat file and creates df out of it for all of the trials of one subject
  # i'm planning on making it more beautiful later

  columns = data[0][1].dtype.names
  df = pd.DataFrame(columns=columns)
  df['Trial'] = np.nan
  df_ = df.copy()
  for i in range(0, data.shape[1]): # for each trial number
    mdtype = data[0][i].dtype
    values = []
    for name in mdtype.names:
      values.append(data[0][i][name])
    values = extra_arrays(values)
    values.append(str(i)) # adding a trial number at the end
    df_.loc[len(df_.index)] = values
  return df_

In [9]:
def extra_arrays(x):
  # elements have nested structure, i.e. arrays inside the arrays
  # below we're getting rid of this depending on the shape of values

  values = []
  for elem in range(0, len(x)):
    var = x[elem]
    var = var[0, 0]
    if var.shape == (1, 1):
      var = var[0, 0]
      values.append(var)
    elif var.shape == (1,):
      values.append(var[0])
    elif var.shape == (1, 5):
      var = var.tolist()[0]
      values.append(var)
    elif var.shape == (5, 1):
      new = []
      for i in var:
        i = i[0]
        new.append(i)
      values.append(new)
    else:
      values.append(var)
  return values

In [10]:
columns_ieeg = data_ieeg['trialInfo'][0, 0][0, 0].dtype.names # name of the variables from trial info
final_ieeg = pd.DataFrame(columns=columns_ieeg)
final_ieeg['Trial'] = np.nan
final_ieeg['Subject'] = np.nan
for subj in range(0, data_ieeg['trialInfo'].shape[1]): # for each subject
  print(subj)
  subj_df = get_to_df(data_ieeg['trialInfo'][0, subj])
  subj_df['Subject'] = subjects_ieeg[subj]
  final_ieeg = pd.concat([final_ieeg, subj_df], axis = 0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [11]:
# D72 subject
one_subject = get_to_df(new_subject1['trialInfo'])
one_more_subject = get_to_df(new_subject2['trialInfo'])

In [12]:
one_subject['Subject'] = 'D72'
one_more_subject['Subject'] = 'D73'

In [13]:
ieeg = pd.concat([final_ieeg, one_subject])

In [14]:
ieeg = pd.concat([ieeg, one_more_subject])

In [18]:
len(ieeg.Subject.unique())

32

## EEG dataset

In [ ]:
data_eeg = eeg['Subject'] # contains trials for each of the subjects
print('Number of subjects in EEG dataset is', data_eeg.shape[1])

Number of subjects in EEG dataset is 29


In [ ]:
subjects_eeg = [] # subject numbers
for i in range(0, data_eeg.shape[1]):
  name = data_eeg[0, i]['Name'][0][0][0]
  subjects_eeg.append(name)

In [ ]:
columns_eeg = data_eeg[0, 0]['trialInfo'][0, 0].dtype.names # name of the variables from trial info
final_eeg = pd.DataFrame(columns=columns_eeg)
final_eeg['Trial'] = np.nan
final_eeg['Subject'] = np.nan
for subj in range(0, data_eeg.shape[1]): # for each subject
  print(subj)
  subj_df = get_to_df(data_eeg[0, subj]['trialInfo'][0, 0])
  subj_df['Subject'] = subjects_eeg[subj]
  final_eeg = pd.concat([final_eeg, subj_df], axis = 0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


In [ ]:
final_eeg.columns

Index(['Trial', 'Subject', 'Block', 'ProbeType', 'ProbeTypeName',
       'StimulusCategory', 'ListenCue', 'ReactionTime', 'KeyCode', 'Response',
       'ResponseCorrect', 'Omission', 'StimulusAudioStart',
       'StimulusAlignedTrigger', 'DelayPeriodOnset', 'MaintenancePeriodOnset',
       'ProbeCueOnset', 'ProbeAudioStart', 'ProbeAlignedTrigger',
       'TriggerValueCondition', 'StimulusSounds_idx', 'StimulusSounds_name',
       'ProbeCategory', 'ProbeSound_idx', 'ProbeSound_name'],
      dtype='object')

In [ ]:
final_eeg.shape

(5568, 25)

In [ ]:
final_eeg.isnull().sum()

Trial                       0
Subject                     0
Block                       0
ProbeType                   0
ProbeTypeName               0
StimulusCategory            0
ListenCue                   0
ReactionTime              100
KeyCode                     0
Response                    0
ResponseCorrect             0
Omission                    0
StimulusAudioStart          0
StimulusAlignedTrigger      0
DelayPeriodOnset            0
MaintenancePeriodOnset      0
ProbeCueOnset               0
ProbeAudioStart             0
ProbeAlignedTrigger         0
TriggerValueCondition       0
StimulusSounds_idx          0
StimulusSounds_name         0
ProbeCategory               0
ProbeSound_idx              0
ProbeSound_name             0
dtype: int64

In [19]:
# saving result
final_eeg.to_csv(
    Path(working_path).joinpath('eeg_all.csv')
)
ieeg.to_csv(
    Path(working_path).joinpath('ieeg_all.csv')
)